In [5]:
from ortools.sat.python import cp_model
import pandas as pd

# Inside/Outside Production

You need to help a manufacturer decide how much of each demanded product should be produced internally and how much should be sourced from outside. Whether a product is made inside or outside, it has an associated cost. A product can consume a given amount of internal resources that have limited capacities. The general objective is to minimize the total production cost while ensuring the company meets the demand exactly or with a certain tolerance.

The manufacturer wants to consider various production decisions to choose the most suitable ones based on the long-term resource availability and uncertain future demand.

Here are examples of the data:

![](https://dmcommunity.org/wp-content/uploads/2025/11/image-15.png)

![](https://dmcommunity.org/wp-content/uploads/2025/11/image-10.png)

In [37]:
Products = {
    "P1": {"Demand": 250, "inside": 6, "outside": 8},
    "P2": {"Demand": 300, "inside": 8, "outside": 9},
    "P3": {"Demand": 200, "inside": 3, "outside": 4}
}

Resource_cap = {
    "R1": 1200,
    "R2": 800,
    "R3": 700
}

Prod_Res_Cap = {
    "P1": {"R1": 5, "R2": 2},
    "P2": {"R1": 4, "R2": 4},
    "P3": {"R1": 3, "R3": 6},
}

In [ ]:
model = cp_model.CpModel()
solver = cp_model.CpSolver()

# Variable number of products chosen 
p = {(i,j): model.NewIntVar(0,Products[i]["Demand"], f'p_{i,j}') for i in Products.keys() for j in ["inside", "outside"]}

# meet demand
for i in Products.keys():
    model.Add(sum(p[(i,j)] for j in ["inside", "outside"]) == Products[i]["Demand"])

# don't exceed internal capacity
for i in Prod_Res_Cap.keys():
    for j in Prod_Res_Cap[i].keys():
        model.Add(p[(i,'inside')]*Prod_Res_Cap[i][j] <= Resource_cap[j])

# minimize costs
model.Minimize(sum(p[i,j]*Products[i][j] for i,j in p.keys()))

In [ ]:
status = solver.Solve(model)
print(f'Status = {solver.StatusName(status)}')

print(f'Objective Value = {solver.objective_value}')

solution = [f'{i,j} = {(solver.value(p[i,j])) }' for i,j in p]
solution

Status = OPTIMAL
Objective Value = 4704.0


["('P1', 'inside') = 240",
 "('P1', 'outside') = 10",
 "('P2', 'inside') = 200",
 "('P2', 'outside') = 100",
 "('P3', 'inside') = 116",
 "('P3', 'outside') = 84"]